In [6]:
import numpy as np
import cv2
import PerImageProcessing
import MaskingProcess
import Evaluation
import random


VID = 0
SING_IMG = 1
INI_PROCESS = 0
SPEED_PROCESS = 1
FINAL_PROCESS = 2

def detection_process(img, mode, name_images, nb_row = 6, sky = 1, vp_on = 1):
    """
    input : list of img to be analyzed in rgb format
    output : not yet defined, prob data + flag set to one if analyzing went smoothly 
    """
    imgs_annotated = []
    imgs_crops_only = []
    stage = INI_PROCESS
    vp_pt = 0
    idx = 0

    if stage == INI_PROCESS: #longer but needed to create initial mask of images 
        print('initial process...')
        hough_img, arr_mask, col_best_mask, vp_pt = PerImageProcessing.Initial_Process(img, nb_row = nb_row, sky = sky)
        if (idx == 0):
            height_original = img.shape[0]
        cv2.imshow('img hough_img: ', cv2.cvtColor(hough_img, cv2.COLOR_RGB2BGR))
        height, _, = arr_mask[0].shape
        stage = SPEED_PROCESS
        print('...speed process...')
    
    if stage == SPEED_PROCESS: #quick, use of ransac 
        img = img[height_original-height:,:,:]
        arr_mask, img_annotated, vp_pt, cond_speed, img_crops_only, pts1, pts2 = PerImageProcessing.speed_process_lines(img, col_best_mask, arr_mask, vp_pt, vp_on=vp_on)
        
        imgs_annotated.append(img_annotated)
        imgs_crops_only.append(img_crops_only)
        r_acc, th_acc,threshold_acc = MaskingProcess.get_r_theta(pts1, pts2)
            
        vp_pt, _ = MaskingProcess.VP_detection(th_acc, r_acc, threshold_acc, stage)
        vp_pt = np.asarray(vp_pt)
                            
   
        if(mode == SING_IMG):
            cv2.imshow('crop only : ', img_crops_only)
            cv2.imshow('img annotated: ', img_annotated)

            cv2.waitKey(0)
        stage = INI_PROCESS

    stage = FINAL_PROCESS


    if stage == FINAL_PROCESS : #save data and evaluate it 
        print('...processing done!')
        if(mode ==SING_IMG):
            crop_only = Evaluation.SaveData(img_crops_only, pts1, pts2)
            cop, cv, dv, v0, array_GT = Evaluation.LoadGroundTruth(name_images, img_annotated)
            score, precision = Evaluation.evaluate_results(cv, dv, v0, array_GT, nb_crop_row=nb_row)
            cv2.imshow('My Results : ', cv2.cvtColor(cop, cv2.COLOR_RGB2BGR))
            cv2.waitKey(0)
            if cv2.waitKey(1) == ord('q'):
                cv2.destroyAllWindows()

    return score #return something en rapport avec le bon acheminement du process

def main_process():
    sky_on = 1
    mode = SING_IMG
    #first, get the name of the files we are going to analyze


    if (mode == SING_IMG): 
        imgs_folder = '/home/roxane/Desktop/M3_2022/Caterra/dataset_straigt_lines' # '/home/roxane/Desktop/M3_2022/USB/Realsense_18-08-2022_10-46-58/' 
        name_images = 'crop_row_256.JPG' #'rgb000.jpg' # crop_row_001, crop_row_020, crop_row_053
    sky_on = 0
    nb_row = 5
    vp_on = 1

    # open and resize images for consistency --> returns img in rgb format
    images = MaskingProcess.obtain_images(name_images,imgs_folder, mode)

    # Main Detection function 
    score = detection_process(img, mode, name_images, nb_row = nb_row, sky = sky_on, vp_on = vp_on)
    print('score : ', score)

        

In [7]:
main_process()

    if (mode == VID):
        #distinguer entre mode video et mode single image?
        imgs_folder = '/home/roxane/Desktop/M3_2022/USB/Realsense_18-08-2022_10-46-58/'
        name_images = MaskingProcess.obtain_name_images(imgs_folder)
        sky_on = 1
        nb_row = 5
        vp_on = 1

image read :  /home/roxane/Desktop/M3_2022/Caterra/dataset_straigt_lines/crop_row_256.JPG
initial process...
the best color is : rgb  [145 180  88] hex :  #91B458
step  1 of  6
step  2 of  6
step  3 of  6
step  4 of  6
step  5 of  6
step  6 of  6
is it outlier = eps max, var 1.3269720143536574 1.1423088376505246
no outliers lines
...speed process...
...processing done!
score :  0.8905115764796386
score :  0.8905115764796386


: 